In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("./data/rev_gtd3.csv", index_col=0)

In [3]:
def clean(df):
    
    df.date = pd.to_datetime(df.date)
    df = df.drop(columns=['eventid', 'provstate', 'city', 'id', 'loc_id', 'region', 'country'])
    df['year'] = df.date.dt.year
    df = df.assign(loc_id=(df['latitude'].astype(str) + '_' + df['longitude'].astype(str)).astype('category').cat.codes)
    df['unique_id'] = df[['loc_id','year']].apply(tuple, axis=1)

    for col in ['elevation', 'DIS_LAKE',
           'DIS_MAJOR_RIVER', 'DIS_OCEAN', 'DIS_RIVER', 'MER1990_40', 'MER1995_40',
           'MER2000_40', 'MER2005_40', 'POPGPW_1990_40', 'POPGPW_1995_40',
           'POPGPW_2000_40', 'POPGPW_2005_40', 'PRECAVNEW80_08', 'TEMPAV_8008']:
        df[col] = df[col].astype(float)
    
    return df
    

In [4]:
def aggregate(df):
    
    df = clean(df)

    df_list = list()
    col_names = list()
    for col in ['attacktype1_txt', 'targtype1_txt',
           'targsubtype1_txt', 'group_name']:
        temp_series = df.groupby('unique_id')[col].nunique()
        df_list.append(temp_series)
        col_names.append(col)
    
    for col in ['nkill', 'nwound']:
        temp_series = df.groupby('unique_id')[col].sum()
        df_list.append(temp_series)
        col_names.append(col)
    
    for col in ['elevation', 'DIS_LAKE',
           'DIS_MAJOR_RIVER', 'DIS_OCEAN', 'DIS_RIVER', 'PRECAVNEW80_08', 'TEMPAV_8008',
           'ethin_div', 'HighRelig', 'ChrCatP', 'ReligCatP', 'year', 'loc_id']:
        temp_series = df.groupby('unique_id')[col].unique().apply(lambda x: x[0])
        df_list.append(temp_series)
        col_names.append(col)

    for col in ['MER{}_40', 'POPGPW_{}_40']:
        if df.year.max() < 1995:
            year = 1990
        elif df.year.max() < 2000:
            year = 1995
        elif df.year.max() < 2005:
            year = 2000
        else:
            year = 2005
        temp_series = df.groupby('unique_id')[col.format(year)].unique().apply(lambda x: x[0])
        df_list.append(temp_series)

    col_names += ['MER_40', 'POPGPW_40']

    final_df = pd.concat(df_list, axis=1, keys=col_names)
    
    final_df['attacked'] = 1
    
    return final_df

In [5]:
final_df = aggregate(df)

In [6]:
final_df.head(100)

,attacktype1_txt,targtype1_txt,targsubtype1_txt,nkill,nwound,elevation,DIS_LAKE,DIS_MAJOR_RIVER,DIS_OCEAN,DIS_RIVER,...,TEMPAV_8008,ethin_div,HighRelig,ChrCatP,ReligCatP,year,loc_id,MER_40,POPGPW_40,attacked
unique_id,,,,,,,,,,,,,,,,,,,,,
"(0, 1984)",1,1,1,8.0,0.0,190.4112,947502.9,749025.9,190411.20,412990.4,...,16.468319,2,Christians,95-100%,95-100%,1984,0,0.547540,544749.27430,1
"(0, 1988)",2,3,3,5.0,0.0,190.4112,947502.9,749025.9,190411.20,412990.4,...,16.468319,2,Christians,95-100%,95-100%,1988,0,0.547540,544749.27430,1
"(0, 1989)",2,5,6,4.0,0.0,190.4112,947502.9,749025.9,190411.20,412990.4,...,16.468319,2,Christians,95-100%,95-100%,1989,0,0.547540,544749.27430,1
"(0, 1990)",1,1,1,5.0,0.0,190.4112,947502.9,749025.9,190411.20,412990.4,...,16.468319,2,Christians,95-100%,95-100%,1990,0,0.547540,544749.27430,1
"(0, 1991)",2,2,2,3.0,1.0,190.4112,947502.9,749025.9,190411.20,412990.4,...,16.468319,2,Christians,95-100%,95-100%,1991,0,0.547540,544749.27430,1
"(0, 1994)",1,1,1,0.0,0.0,190.4112,947502.9,749025.9,190411.20,412990.4,...,16.468319,2,Christians,95-100%,95-100%,1994,0,0.547540,544749.27430,1
"(1, 1990)",1,1,1,4.0,0.0,86.6291,1027299.0,810529.3,86629.05,468072.8,...,7.889152,1,Christians,95-100%,95-100%,1990,1,0.404205,412964.59300,1
"(2, 1980)",1,3,4,0.0,0.0,314.6529,719260.3,771925.3,314652.90,489996.7,...,28.092960,1,Christians,95-100%,95-100%,1980,2,0.069681,37318.55288,1
"(2, 1981)",1,1,1,0.0,0.0,314.6529,719260.3,771925.3,314652.90,489996.7,...,28.092960,1,Christians,95-100%,95-100%,1981,2,0.069681,37318.55288,1


In [8]:
df = clean(df)

In [75]:
def sum_past_k_year_data(df, col, loc_id, year, k):
    
    agg_num = df.groupby(['loc_id','year']).sum().loc[loc_id].loc[year - k + 1:year].loc[:, col].sum()
    
    return agg_num


def count_past_k_year_data(df, col, loc_id, year, k):
    
    count = len(set(df.groupby(['loc_id', 'year', col]).count().loc[loc_id].loc[year - k + 1:year].index.get_level_values(col).values.tolist()))
    
    return count

In [76]:
sum_past_k_year_data(df, 'nkill', 0, 2015, 100)

25.0

In [77]:
count_past_k_year_data(df, 'targsubtype1_txt', 0, 2015, 100)

12

In [ ]:
k = 5
arg = [df, 'nkill', row['loc_id'], row['year'], 5]
final_df['nkill_testing'] = final_df.apply(lambda row : sum_past_k_year_data(*arg), axis=1)

In [ ]:
k = 5
arg = [df, col, row['loc_id'], row['year'], k]
for col in ['nkill', 'nwound']:
        col_name = col + '_past_{}'.format(k)
        final_df[col_name] = final_df.apply(lambda row : sum_past_k_year_data(*arg), axis=1)
        
for col in ['attacktype1_txt', 'targtype1_txt', 'targsubtype1_txt']:
        col_name = col + '_past_{}'.format(k)
        final_df[col_name] = final_df.apply(lambda row : count_past_k_year_data(*arg), axis=1)

In [ ]:
final_df.head()